# Redes Neuronales - Trabajo Práctico N° 2 - Ejercicio 2
# Notebook #7: Búsqueda de hiperparámetros usando HyperOpt

## Integrantes del grupo
* Gaytan, Joaquín Oscar
* Kammann, Lucas Agustín

# Fuentes

### Link: https://www.dlology.com/blog/how-to-do-hyperparameter-search-with-baysian-optimization-for-keras-model/
Explicación de cómo utilizar el método de optimización bayesiana para buscar los hiper parámetros del modelo para resolver el problema.

### Link: https://www.kaggle.com/prashant111/bayesian-optimization-using-hyperopt
Ahora viendo cómo resolverlo utilizando la librería de **hyperopt**.

# 1. Cargando base de datos

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
import importlib

In [3]:
import sys

In [4]:
sys.path.insert(0, '..')

In [5]:
sys.path.insert(0, '../..')

In [6]:
# Read the database from the .csv file into a pandas dataframe
df = pd.read_csv('../../databases/insurance.csv')

# 2. Preprocesamiento de los datos

In [7]:
from sklearn import preprocessing

In [8]:
from src import helper
importlib.reload(helper);

## 2.1. Codificación de variables no numéricas o categóricas

In [9]:
# Create a label encoder for the sex variable or feature and create a new column in the dataframe 
# with the encoded version of the gender
sex_encoder = preprocessing.LabelEncoder()
sex_encoder.fit(df['sex'])
df['sex-encoded'] = sex_encoder.transform(df['sex'])

In [10]:
# Create a label encoder for the smoker variable or feature and create a new column in the dataframe
# with the encoded version of the smoker
smoker_encoder = preprocessing.LabelEncoder()
smoker_encoder.fit(df['smoker'])
df['smoker-encoded'] = smoker_encoder.transform(df['smoker'])

In [11]:
# Create a label encoder for the region variable or feature and create a new column in the dataframe
# with the encoded version of the region
region_encoder = preprocessing.LabelEncoder()
region_encoder.fit(df['region'])
df['region-encoded'] = region_encoder.transform(df['region'])

## 2.2. Eliminando outliers
A partir del análisis realizado sobre la base de datos sobre la cual se entrenan los modelos, se detectaron outliers en la variable del índice de masa corporal, se decide remover estos datos por completo ya que no fue necesario tener en cuenta una estrategia para corregir datos incompletos o incorrectos y el impacto que tiene sobre la totalidad de datos es menos del 1%. Entonces, se eliminan los casos con outliers del BMI.

In [12]:
# Remove outliers by setting NaN on those rows at the column of BMI
helper.remove_outliers(df, 'bmi')

# Remove NaN values from the dataframe
df = df.dropna()

## 2.3. Filtrado de variables

In [13]:
# Filtering or removing of non desired variables
df_x = df[['age', 'bmi', 'smoker-encoded', 'children', 'sex-encoded', 'region-encoded']]
df_y = df['charges']

# 3. Multilayer Perceptron

In [14]:
import hyperopt

In [15]:
from hyperopt.pyll import scope

In [16]:
from src.ej2 import mlp_helper
importlib.reload(mlp_helper);

## 3.1. Creación de la función **objetivo**
Creamos la función **objetivo** la cual recibe los parámetros a partir de los cuales se crea el modelo, se lo entrena y se evalúa su performance.

In [17]:
def objective(kwargs):
    """ Objective function for the hyperopt algorithm. """
    
    # Setting some fixed values
    kwargs['tensorboard_on'] = False
    kwargs['summary_on'] = False
    kwargs['epochs'] = 500
    kwargs['batch_size'] = 32
    kwargs['optimizer'] = 'adam'
    kwargs['beta_1'] = 0.9
    kwargs['beta_2'] = 0.999
    
    # Running the model with the given hyperparameters and retrieving the test set performance
    # WARNING! We're using the same valid set for both valid and test, but can be ignored, it does not affect
    # because it was something we had to do to reuse the function...
    _, valid_metrics, _ = mlp_helper.run_model_with_kfold(df_x, df_y, test_size=0.2, n_splits=5, random_state=15, **kwargs)
    return {'loss': round(valid_metrics.mean(), 2), 'status': hyperopt.STATUS_OK}

## 3.2. Creación del **espacio** de hiperparámetros
Creamos el **espacio** en el cual se encuentran cada uno de los hiperparámetros que queremos probar, es decir, cuáles son los hiperparámetros y qué valores pueden tomar en cada prueba a realizar por el optimizador de hiperparámetros.

In [18]:
space = {
    'use_batch_normalization': hyperopt.hp.choice('use_batch_normalization', [False, True]),
    'regularizer': hyperopt.hp.choice('regularizer', ['l2', None]),
    'learning_rate': hyperopt.hp.choice('learning_rate', [1.0, 0.5, 0.1, 0.01]),
    'hidden_layers': scope.int(hyperopt.hp.quniform('hidden_layers', 0, 10, 1)),
    'units_per_layer': scope.int(hyperopt.hp.quniform('units_per_layer', 2, 50, 1)),
    'hidden_layer_activation': hyperopt.hp.choice('hidden_layer_activation', ['relu', 'elu'])
}

## 3.3. Selección del **algoritmo** de optimización
Se puede elegir un **algoritmo** de optimización para el proceso de búsqueda de los mejores hiperparámetros dentro del espacio consignado acorde a los resultados que se van obteniendo de la función objetivo.

In [19]:
algorithm = hyperopt.tpe.suggest

## 3.4. Ejecutando la búsqueda
Se ejecuta la búsqueda de los mejores hiperparámetros.

In [20]:
trials = hyperopt.base.Trials()

In [21]:
best = hyperopt.fmin(objective, space, algo=algorithm, max_evals=100, trials=trials)

100%|█████████████████████████████████████████████████████| 100/100 [15:55:21<00:00, 573.21s/trial, best loss: 1549.93]


In [21]:
hyperopt.space_eval(space, best)

{'hidden_layer_activation': 'elu',
 'hidden_layers': 6,
 'learning_rate': 0.1,
 'regularizer': None,
 'units_per_layer': 38,
 'use_batch_normalization': False}

In [22]:
# Run model
train_maes, valid_maes, test_maes = mlp_helper.run_model_with_kfold(df_x, df_y, test_size=0.2, n_splits=5, random_state=15,
                                                                    epochs=500,
                                                                    batch_size=32,
                                                                    optimizer='adam',
                                                                    beta_1=0.9,
                                                                    beta_2=0.999,
                                                                    tag='hyperopt',
                                                                    **hyperopt.space_eval(space, best),
                                                                    tensorboard_on=False,
                                                                    summary_on=False
                                                                   )

# Inform results
mae_train = round(train_maes.mean(), 2)
mae_valid = round(valid_maes.mean(), 2)
mae_test = round(test_maes.mean(), 2)
print(f'[MAE] Train: {mae_train} Valid: {mae_valid} Test: {mae_test}')

[MAE] Train: 1489.45 Valid: 1583.39 Test: 1725.39
